### 절대 모멘텀
- 파생변수 'STD-YM' 생성 -> index에서 년-월 추출하여 대입
- 'STD-YM' 별 마지막날의 데이터를 month_last_df 저장
- 전월의 수정주가 값을 가진 파생변수를 생성 
- 전년도의 수정주가 값을 가진 파생변수를 생성
- 전월의 데이터와 전년도의 데이터를 기준으로 하여 거래 내역 생성
- 수익율 계산

In [23]:
import pandas as pd 
import numpy as np 
from datetime import datetime

In [24]:
df = pd.read_csv("../../csv/MSFT.csv", index_col='Date')

In [25]:
df.head(1)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1986-03-13,0.088542,0.101563,0.088542,0.097222,0.069996,1031788800


In [26]:
# index를 시계열로 변경 
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

In [27]:
# index의 데이터에서 년-월 추출하여 새로운 파생변수 STD-YM에 대입
df['STD-YM'] = df.index.strftime('%Y-%m')

In [28]:
df.iloc[10:20]

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1986-03-27,0.094618,0.096354,0.094618,0.096354,0.069371,16848000,1986-03
1986-03-31,0.096354,0.096354,0.093750,0.095486,0.068746,12873600,1986-03
1986-04-01,0.095486,0.095486,0.094618,0.094618,0.068121,11088000,1986-04
1986-04-02,0.094618,0.097222,0.094618,0.095486,0.068746,27014400,1986-04
1986-04-03,0.096354,0.098958,0.096354,0.096354,0.069371,23040000,1986-04
1986-04-04,0.096354,0.097222,0.096354,0.096354,0.069371,26582400,1986-04
1986-04-07,0.096354,0.097222,0.092882,0.094618,0.068121,16560000,1986-04
1986-04-08,0.094618,0.097222,0.094618,0.095486,0.068746,10252800,1986-04
1986-04-09,0.095486,0.098090,0.095486,0.097222,0.069996,12153600,1986-04


In [29]:
# 월의 말일의 조건식
# 다음 행의 STD-YM과 오늘의 STD-YM의 값이 다른경우
flag = df['STD-YM'] != df.shift(-1)['STD-YM']
df.loc[flag,]

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1986-03-31,0.096354,0.096354,0.093750,0.095486,0.068746,12873600,1986-03
1986-04-30,0.114583,0.115451,0.109375,0.111979,0.080620,30902400,1986-04
1986-05-30,0.118056,0.123264,0.118056,0.121528,0.087495,27072000,1986-05
1986-06-30,0.103299,0.109375,0.102431,0.106771,0.076870,62352000,1986-06
1986-07-31,0.099826,0.100694,0.098958,0.098958,0.071246,15638400,1986-07
...,...,...,...,...,...,...,...
2019-02-28,112.040001,112.879997,111.730003,112.029999,111.616837,29083900,2019-02
2019-03-29,118.070000,118.320000,116.959999,117.940002,117.505043,25399800,2019-03
2019-04-30,129.809998,130.699997,129.389999,130.600006,130.118362,24166500,2019-04


In [30]:
# 필터를 이용하여 마지막날의 데이터를 추출 
_list =  df['STD-YM'].unique()

month_last_df = pd.DataFrame()

for i in _list:
    flag = df['STD-YM'] == i
    data = df.loc[flag,].tail(1)
    month_last_df = pd.concat([month_last_df, data], axis=0)

In [31]:
month_last_df

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1986-03-31,0.096354,0.096354,0.093750,0.095486,0.068746,12873600,1986-03
1986-04-30,0.114583,0.115451,0.109375,0.111979,0.080620,30902400,1986-04
1986-05-30,0.118056,0.123264,0.118056,0.121528,0.087495,27072000,1986-05
1986-06-30,0.103299,0.109375,0.102431,0.106771,0.076870,62352000,1986-06
1986-07-31,0.099826,0.100694,0.098958,0.098958,0.071246,15638400,1986-07
...,...,...,...,...,...,...,...
2019-02-28,112.040001,112.879997,111.730003,112.029999,111.616837,29083900,2019-02
2019-03-29,118.070000,118.320000,116.959999,117.940002,117.505043,25399800,2019-03
2019-04-30,129.809998,130.699997,129.389999,130.600006,130.118362,24166500,2019-04


In [45]:
# 전월의 수정 주가를 BF_1M 파생변수에 대입 
# 전년도의 수정주가를 BF_12M 파생변수에 대입 
# 두개의 파생변수는 결측치는 0으로 대체
month_last_df['BF_1M'] = month_last_df['Adj Close'].shift(1).fillna(0)
month_last_df['BF_12M'] = month_last_df['Adj Close'].shift(12).fillna(0)

- 거래 내역 생성 
    - (전월의 수정주가 / 전년도의 수정주가) - 1 의 값이 0보다 크고 무한대가 아닌 경우

In [33]:
df['trade'] = ""

for i in month_last_df.index:
    signal = ""

    # 절대 모멘텀 계산식 
    momentum_index = month_last_df.loc[i, 'BF_1M'] /\
        month_last_df.loc[i, 'BF_12M'] - 1
    flag = True if ( (momentum_index > 0) & (momentum_index != np.inf) ) \
        else False
    
    if flag:
        signal = "buy"
    
    print(f"날짜 : {i}, 모멘텀 인덱스 : {momentum_index}, flag : {flag}, signal : {signal}")
    df.loc[i:, 'trade'] = signal


날짜 : 1986-03-31 00:00:00, 모멘텀 인덱스 : nan, flag : False, signal : 
날짜 : 1986-04-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-05-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-06-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-07-31 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-08-29 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-09-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-10-31 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-11-28 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-12-31 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1987-01-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1987-02-27 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1987-03-31 00:00:00, 모멘텀 인덱스 : 1.7909114712128704, flag : True, signal : buy
날짜 : 1987-04-30 00:00:00, 모멘텀 인덱스 : 2.0000124038700076, flag : True, signal : buy
날짜 : 1987-05-29 00:00:00, 모멘텀 인덱스 : 1.971415509457683, f

C:\Users\82105\AppData\Local\Temp\ipykernel_148452\238936273.py:7: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index = month_last_df.loc[i, 'BF_1M'] /\
C:\Users\82105\AppData\Local\Temp\ipykernel_148452\238936273.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = month_last_df.loc[i, 'BF_1M'] /\


In [34]:
df['trade'].value_counts()

trade
buy    6005
       2384
Name: count, dtype: int64

In [35]:
df['rtn'] = 1
for i in df.index:
    # 매수 조건식 (전날의 trade가 "" 오늘의 trade가 "buy")
    if (df.shift().loc[i, 'trade'] == "") & (df.loc[i, 'trade'] == "buy"):
        buy = df.loc[i, 'Adj Close']
        print(f"매수일 : {i}, 매수가 : {buy}")
    # 매도 조건식 (전날의 trade가 "buy" 오늘의 trade가 "")
    elif (df.shift().loc[i, 'trade'] == "buy") & (df.loc[i, 'trade'] == ""):
        sell = df.loc[i, 'Adj Close']
        rtn = sell / buy
        df.loc[i, 'rtn'] = rtn
        print(f"매도일 : {i}, 매도가 : {sell}, 수익율 : {rtn}")

매수일 : 1987-03-31 00:00:00, 매수가 : 0.241861


C:\Users\82105\AppData\Local\Temp\ipykernel_148452\2680049231.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.1989655215185582' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'rtn'] = rtn


매도일 : 1988-05-31 00:00:00, 매도가 : 0.289983, 수익율 : 1.1989655215185582
매수일 : 1988-06-30 00:00:00, 매수가 : 0.33498
매도일 : 1988-09-30 00:00:00, 매도가 : 0.261234, 수익율 : 0.7798495432563138
매수일 : 1988-10-31 00:00:00, 매수가 : 0.244985
매도일 : 1988-12-30 00:00:00, 매도가 : 0.266234, 수익율 : 1.0867359226075066
매수일 : 1989-02-28 00:00:00, 매수가 : 0.297482
매도일 : 1989-04-28 00:00:00, 매도가 : 0.279358, 수익율 : 0.939075305396629
매수일 : 1989-08-31 00:00:00, 매수가 : 0.293733
매도일 : 1993-08-31 00:00:00, 매도가 : 1.690211, 수익율 : 5.754242798732181
매수일 : 1994-02-28 00:00:00, 매수가 : 1.85614
매도일 : 1994-03-31 00:00:00, 매도가 : 1.906761, 수익율 : 1.0272721885202625
매수일 : 1994-06-30 00:00:00, 매수가 : 2.322986
매도일 : 2000-03-31 00:00:00, 매도가 : 38.247726, 수익율 : 16.464897334723496
매수일 : 2000-04-28 00:00:00, 매수가 : 25.108511
매도일 : 2000-05-31 00:00:00, 매도가 : 22.521158, 수익율 : 0.8969531486753635
매수일 : 2001-05-31 00:00:00, 매수가 : 24.903316
매도일 : 2001-06-29 00:00:00, 매도가 : 26.278425, 수익율 : 1.055217907526853
매수일 : 2001-07-31 00:00:00, 매수가 : 23.826982
매도일 : 200

In [36]:
df['acc_rtn'] = df['rtn'].cumprod()

In [37]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume,STD-YM,trade,rtn,acc_rtn
Date,,,,,,,,,,
2019-06-18,134.190002,135.240005,133.570007,135.160004,135.160004,25934500,2019-06,buy,1.0,72.288746
2019-06-19,135.000000,135.929993,133.809998,135.690002,135.690002,23744400,2019-06,buy,1.0,72.288746
2019-06-20,137.449997,137.660004,135.720001,136.949997,136.949997,33042600,2019-06,buy,1.0,72.288746
2019-06-21,136.580002,137.729996,136.460007,136.970001,136.970001,36348400,2019-06,buy,1.0,72.288746
2019-06-24,137.000000,138.389999,137.020004,138.289993,138.289993,9131553,2019-06,buy,1.0,72.288746


### 절대 모멘텀 함수화
1. STD-YM 생성하는 함수 
    - 매개변수 2개 (데이터프레임, 기준이 되는 컬럼)
    - 데이터프레임의 복사본을 생성
    - 컬럼에 Date가 포함되어있는지 확인하고 컬럼에 Date 존재한다면 Date를 인덱스로 변환
    - 인덱스를 시계열 데이터로 변경 
    - 데이터중 결측치나 무한대 값을 제외 
    - 기준이 되는 컬럼을 제외한 나머지 컬럼은 모두 삭제
    - 'STD-YM' 컬럼을 생성하여 index에서 '년-월'데이터를 추출하여 대입 
    - 수정이 된 데이터프레임을 리턴

In [46]:
def create_YM(
        _df,
        _col= 'Adj Close'
):
    df=_df.copy()
    # Date가 컬럼에 포함되어있는가?
    if 'Date' in df.columns:
        # 포함되어있다면 Date를 인덱스로 변환
        df.set_index('Date',inplace=True)
    # 인덱스를 시계열 데이터로 변경
        df.index=pd.to_datetime(df.index, format='%Y-%m-%d')
        # 결측치, 무한대 데이터를 제거 기준이되는 컬럼만 두고 나머지 모두 제거
        flag=df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
        df=df.loc[~flag, [_col]]
        # 파생변수 std-ym 생성
        df['STD-YM']=df.index.strftime('%Y-%m')

        return df

In [47]:
df=pd.read_csv('../../csv/AAPL.csv')

In [49]:
ym_df=create_YM(df)

- 두번째 함수생성
    - 매개변수5개(데이터프레임, 시작시간, 종료시간, 모멘텀기간, 기준시점)
    - 시작시간의 기본값은 2010-01-01
    - 종료시간의 기본값은 오늘 날짜
    - 모멘텀 기간의 기본값은 12
    - 기준시점의 기본값은 1이고 0으로 변경시 월초의 데이터로 구성
    - 기준시점의 값에 따라 월말 데이터만 모아서 새로운 데이터 프레임 생성
    - 생성된 데이터프레임을 기준으로 BF1 컬럼을 생성하여 전월의 데이터를 대입
    - BF2 컬럼을 생성하여 모멘텀 기간(6-> 6개월 전의 데이터임) 전의 데이터를 대입
    - 결측치는 0으로 대체
    - 데이터프레임을 시작시간과 종료시간으로 데이터 필터링
    - 결과 리턴

In [58]:
def create_month(
        _df,
        _start='2010-01-01',
        _end=datetime.now(),
        _momentum=12,
        _select=1
):
    if _select == 1:
    # 월말의 데이터들을 새로운 데이터 프레임으로 생성
    # 현재 행의 년-월과 다음 행의 년-월이 다른 경우
        flag =_df['STD-YM'] !=_df.shift(-1)['STD-YM']
    elif _select == 0:
        flag=_df['STD-YM'] != _df.shift()['STD-YM']
    else:
        return "_select 인자는 0과 1이 가능하다"
    col=_df.columns[0]
    df=_df.loc[flag, ]
    # 기준이 되는 컬럼의 데이터를 인덱스 1씩 이동하고 결측치는 0으로 대체
    df['BF1']=df.shift()[col].fillna(0)
    df['BF2']=df.shift(_momentum)[col].fillna(0)
    # 시작시간부터 종료시간까지 데이터를 필터링
    start=datetime.strptime(_start, '%Y-%m-%d')
    if type(_end)=='str':
        end=datetime.strptime(_end,'%Y-%m-%d')
    else:
        end=_end
    df=df.loc[start:end, ]
    return df

In [60]:
month_df=create_month(ym_df, _select=0)

C:\Users\82105\AppData\Local\Temp\ipykernel_148452\3845485724.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BF1']=df.shift()[col].fillna(0)
C:\Users\82105\AppData\Local\Temp\ipykernel_148452\3845485724.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BF2']=df.shift(_momentum)[col].fillna(0)


- 세번째 함수
    - 매개변수 3개(1번 함수결과인 데이터프레임(_df1), 2번 함수의 결과인 데이터프레임(_df2), 모멘텀 스코어)
    - _df1에는 trade 컬럼을 생성 "" 대입
    - _df1 rtn 컬럼을 생성 1 대입
    - _df2를 이용하여 momentem_index를 구하고 _df1에 거래내역 대입
    - 거래 내역을 이용하여 수익을 계산
    - acc_rtn 컬럼을 생성 누적 수입을 대입
    - _df1, 총 누적수익을 리턴

In [68]:
def create_rtn(_df1, _df2, _score=1):
    # _df1에 파생변수 2개 생성
    _df1['trade']=''
    _df2['rtn']=1

    # _df2 데이터를 이용하여 momentem_index를 계산하고 거래내역 추가
    for i in _df2.index:
        signal=""

        # 절대 모멘텀 계산
        momentum_index=_df2.loc[i, 'BF1'] / _df2.loc[i, 'BF2']-_score

        # 모멘텀 인덱스가 무한대가 아니고 0보다 큰 경우
        flag=(momentum_index>0) & (momentum_index != np.inf)

        if flag:
            signal = 'buy'

        _df1.loc[i:,'trade']=signal
        print(f"날짜 : {i}, 모멘텀 인덱스: {momentum_index}, flag:{flag}, signal:{signal}")

    # 수익 계산
    col=_df1.columns[0]

    for i in _df1.index:
        # 구매한 날의 조건식(전날의 trade가 "" 오늘의 trade가 "buy")
        if(_df1.shift().loc[i, 'trade']=="")&(_df1.loc[i, 'trade']=="buy"):
            buy=_df1.loc[i, col]
            print(f"매수일:{i}, 매수가:{buy}")
        # 판매한 날의 조건식(전날의 trade가 "buy" 오늘의 trade가 "")
        elif (_df1.shift().loc[i, 'trade']=="buy")&(_df1.loc[i, 'trade']==""):
            sel1=_df1.loc[i, col]
            rtn=sell/buy
            _df1.loc[i, 'rtn']=rtn
            print(f"매도일:{i}, 매도가:{sell}, 수익률:{rtn}")
    # 누적수익률계산
    _df1['acc_rtn']=_df1['rtn'].cumprod()

    # 총 누적수익을 변수에 대입
    acc_rtn=_df1.iloc[-1, ]['acc_rtn']

    return _df1, acc_rtn

In [69]:
create_rtn(ym_df, month_df)

날짜 : 2010-01-04 00:00:00, 모멘텀 인덱스: 1.1704684764787037, flag:True, signal:buy
날짜 : 2010-02-01 00:00:00, 모멘텀 인덱스: 1.3386514367584996, flag:True, signal:buy
날짜 : 2010-03-01 00:00:00, 모멘텀 인덱스: 1.2143503382467578, flag:True, signal:buy
날짜 : 2010-04-01 00:00:00, 모멘텀 인덱스: 0.9228079513432723, flag:True, signal:buy
날짜 : 2010-05-03 00:00:00, 모멘텀 인덱스: 0.8545267482992649, flag:True, signal:buy
날짜 : 2010-06-01 00:00:00, 모멘텀 인덱스: 0.9113741463644562, flag:True, signal:buy
날짜 : 2010-07-01 00:00:00, 모멘텀 인덱스: 0.826156002181849, flag:True, signal:buy
날짜 : 2010-08-02 00:00:00, 모멘텀 인덱스: 0.492999881218835, flag:True, signal:buy
날짜 : 2010-09-01 00:00:00, 모멘텀 인덱스: 0.5840892661901054, flag:True, signal:buy
날짜 : 2010-10-01 00:00:00, 모멘텀 인덱스: 0.38410945262421703, flag:True, signal:buy
날짜 : 2010-11-01 00:00:00, 모멘텀 인덱스: 0.49236669736674377, flag:True, signal:buy
날짜 : 2010-12-01 00:00:00, 모멘텀 인덱스: 0.5442960562442314, flag:True, signal:buy
날짜 : 2011-01-03 00:00:00, 모멘텀 인덱스: 0.4784358088320484, flag:True, signal:buy

(             Adj Close   STD-YM trade  rtn  acc_rtn
 Date                                               
 1980-12-12    0.410525  1980-12        NaN      NaN
 1980-12-15    0.389106  1980-12        NaN      NaN
 1980-12-16    0.360548  1980-12        NaN      NaN
 1980-12-17    0.369472  1980-12        NaN      NaN
 1980-12-18    0.380182  1980-12        NaN      NaN
 ...                ...      ...   ...  ...      ...
 2019-06-18  198.449997  2019-06   buy  NaN      NaN
 2019-06-19  197.869995  2019-06   buy  NaN      NaN
 2019-06-20  199.460007  2019-06   buy  NaN      NaN
 2019-06-21  198.779999  2019-06   buy  NaN      NaN
 2019-06-24  199.169998  2019-06   buy  NaN      NaN
 
 [9713 rows x 5 columns],
 nan)

In [1]:
import momentum as mm

In [2]:
import yfinance as yf

In [3]:
df=yf.download('TSLA', start='2015-01-01')

[*********************100%%**********************]  1 of 1 completed


In [4]:
ym_df=mm.create_YM(df)

In [5]:
ym_df.head()

,Adj Close,STD-YM
Date,,
2015-01-02,14.620667,2015-01
2015-01-05,14.006000,2015-01
2015-01-06,14.085333,2015-01
2015-01-07,14.063333,2015-01
2015-01-08,14.041333,2015-01


In [6]:
month_df=mm.create_month(ym_df, _start='2015-01-01')
month_df.head()

c:\Users\82105\Documents\GitHub\seoyj\python\20240227\momentum.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BF1']=df.shift()[col].fillna(0)
c:\Users\82105\Documents\GitHub\seoyj\python\20240227\momentum.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BF2']=df.shift(_momentum)[col].fillna(0)


,Adj Close,STD-YM,BF1,BF2
Date,,,,
2015-01-30,13.573333,2015-01,0.000000,0.0
2015-02-27,13.556000,2015-02,13.573333,0.0
2015-03-31,12.584667,2015-03,13.556000,0.0
2015-04-30,15.070000,2015-04,12.584667,0.0
2015-05-29,16.719999,2015-05,15.070000,0.0


In [7]:
result, acc_rtn = mm.create_rtn(ym_df, month_df)

c:\Users\82105\Documents\GitHub\seoyj\python\20240227\momentum.py:68: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index=_df2.loc[i, 'BF1'] / _df2.loc[i, 'BF2']-_score
c:\Users\82105\Documents\GitHub\seoyj\python\20240227\momentum.py:68: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index=_df2.loc[i, 'BF1'] / _df2.loc[i, 'BF2']-_score


날짜 : 2015-01-30 00:00:00, 모멘텀 인덱스: nan, flag:False, signal:
날짜 : 2015-02-27 00:00:00, 모멘텀 인덱스: inf, flag:False, signal:
날짜 : 2015-03-31 00:00:00, 모멘텀 인덱스: inf, flag:False, signal:
날짜 : 2015-04-30 00:00:00, 모멘텀 인덱스: inf, flag:False, signal:
날짜 : 2015-05-29 00:00:00, 모멘텀 인덱스: inf, flag:False, signal:
날짜 : 2015-06-30 00:00:00, 모멘텀 인덱스: inf, flag:False, signal:
날짜 : 2015-07-31 00:00:00, 모멘텀 인덱스: inf, flag:False, signal:
날짜 : 2015-08-31 00:00:00, 모멘텀 인덱스: inf, flag:False, signal:
날짜 : 2015-09-30 00:00:00, 모멘텀 인덱스: inf, flag:False, signal:
날짜 : 2015-10-30 00:00:00, 모멘텀 인덱스: inf, flag:False, signal:
날짜 : 2015-11-30 00:00:00, 모멘텀 인덱스: inf, flag:False, signal:
날짜 : 2015-12-31 00:00:00, 모멘텀 인덱스: inf, flag:False, signal:
날짜 : 2016-01-29 00:00:00, 모멘텀 인덱스: 0.1788311554451023, flag:True, signal:buy
날짜 : 2016-02-29 00:00:00, 모멘텀 인덱스: -0.05970292580194192, flag:False, signal:
날짜 : 2016-03-31 00:00:00, 모멘텀 인덱스: 0.016739870778827992, flag:True, signal:buy
날짜 : 2016-04-29 00:00:00, 모멘텀 인덱스: 0.0164565461

매도일:2016-02-29 00:00:00, 매도가:12.795332908630371, 수익률:1.0038179392869095
매수일:2016-03-31 00:00:00, 매수가:15.317999839782715
매도일:2016-05-31 00:00:00, 매도가:14.881999969482422, 수익률:0.9715367623148847
매수일:2017-01-31 00:00:00, 매수가:16.795333862304688
매도일:2018-04-30 00:00:00, 매도가:19.593332290649414, 수익률:1.166593796305802
매수일:2018-07-31 00:00:00, 매수가:19.875999450683594
매도일:2018-08-31 00:00:00, 매도가:20.110666275024414, 수익률:1.0118065421023519
매수일:2018-11-30 00:00:00, 매수가:23.365333557128906
매도일:2019-01-31 00:00:00, 매도가:20.468000411987305, 수익률:0.8759986396916809
매수일:2019-03-29 00:00:00, 매수가:18.657333374023438
매도일:2019-04-30 00:00:00, 매도가:15.912667274475098, 수익률:0.8528907617971959
매수일:2020-01-31 00:00:00, 매수가:43.371334075927734
매도일:2022-07-29 00:00:00, 매도가:297.1499938964844, 수익률:6.85129937152223
매수일:2022-08-31 00:00:00, 매수가:275.6099853515625
매도일:2022-10-31 00:00:00, 매도가:227.5399932861328, 수익률:0.8255868995308259
매수일:2023-10-31 00:00:00, 매수가:200.83999633789062
매도일:2024-02-27 00:00:00, 매도가:199.7299957275390

In [8]:
acc_rtn

4.837901309487323